## PySpark SetUp

In [2]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
# install Python Reddit API Wrapper for getting external data Karma
%pip install pyspark==3.2.0
%pip install praw


# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import IntegerType
import pandas as pd
import praw

In [3]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f72b13fd-d8b2-4cc3-95d5-919294ce4856;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 404ms :: artifacts dl 34ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

3.2.0


## Submission



In [4]:
%%time
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_data_submissions = f"project/submissions/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_submissions}"
print(f"reading submissions from {s3_path}")
submissions = spark.read.parquet(s3_path, header=True)
print(f"shape of the submissions dataframe is {submissions.count():,}x{len(submissions.columns)}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading submissions from s3a://sagemaker-us-east-1-861795727138/project/submissions/yyyy=*


23/11/10 05:21:28 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/11/10 05:21:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


shape of the submissions dataframe is 95,932x68
CPU times: user 2.79 s, sys: 269 ms, total: 3.06 s
Wall time: 6min 29s


In [5]:
sampled_submissions = submissions.sample(withReplacement=False, fraction=0.1, seed=123)
sampled_submissions.cache()

DataFrame[adserver_click_url: string, adserver_imp_pixel: string, archived: boolean, author: string, author_cakeday: boolean, author_flair_css_class: string, author_flair_text: string, author_id: string, brand_safe: boolean, contest_mode: boolean, created_utc: timestamp, crosspost_parent: string, crosspost_parent_list: array<struct<approved_at_utc:string,approved_by:string,archived:boolean,author:string,author_flair_css_class:string,author_flair_text:string,banned_at_utc:string,banned_by:string,brand_safe:boolean,can_gild:boolean,can_mod_post:boolean,clicked:boolean,contest_mode:boolean,created:double,created_utc:double,distinguished:string,domain:string,downs:bigint,edited:boolean,gilded:bigint,hidden:boolean,hide_score:boolean,id:string,is_crosspostable:boolean,is_reddit_media_domain:boolean,is_self:boolean,is_video:boolean,likes:string,link_flair_css_class:string,link_flair_text:string,locked:boolean,media:string,mod_reports:array<string>,name:string,num_comments:bigint,num_crosspos

## Comment


In [6]:
%%time
import sagemaker
session = sagemaker.Session()
bucket = session.default_bucket()
output_prefix_data_comments = "project/comments/yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
#s3_path = "s3a://sagemaker-us-east-1-038932893404/project/comments/yyyy=2021/part-00000-90796409-5783-4705-92c0-27c27eda8c4c-c000.snappy.parquet"
print(f"reading comments from {s3_path}")
comments = spark.read.parquet(s3_path, header=True)
print(f"shape of the comments dataframe is {comments.count():,}x{len(comments.columns)}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading comments from s3a://sagemaker-us-east-1-861795727138/project/comments/yyyy=*


shape of the comments dataframe is 1,053,280x21
CPU times: user 433 ms, sys: 65.1 ms, total: 498 ms
Wall time: 11min 6s


In [7]:
sampled_comments = comments.sample(withReplacement=False, fraction=0.1, seed=123)
sampled_comments.cache()

DataFrame[author: string, author_cakeday: boolean, author_flair_css_class: string, author_flair_text: string, body: string, can_gild: boolean, controversiality: bigint, created_utc: timestamp, distinguished: string, edited: string, gilded: bigint, id: string, is_submitter: boolean, link_id: string, parent_id: string, permalink: string, retrieved_on: timestamp, score: bigint, stickied: boolean, subreddit: string, subreddit_id: string]

## PRAW

In [10]:
#Get posting karma of the creator from Python Reddit API Wrapper
def udf_get_karma(author_str):
    #Reddit API Credential
    client_id = 'UViHJJ217cLodAY9AraMWg'
    client_secret = 'dvpLKVfv7ItyQhHkYPPOK6o_ssI63w'
    user_agent = 'android:com.example.myredditapp:v1.2.3 (by u/sf_suzyfeng)'
    #Reddit Username
    username = 'sf_suzyfeng'
    #Initialize a Reddit instance
    reddit = praw.Reddit(client_id=client_id,client_secret=client_secret,user_agent=user_agent)
    try:
        redditor= reddit.redditor(author_str)
        link_karma= redditor.link_karma
    #if the account was deleted or ..., set Karma to 0 
    except:
        link_karma= 0
    return link_karma

#Creter udf to retrieve Karma 
udf_karma = F.udf(udf_get_karma, IntegerType())

## Top 10 creators 


In [11]:
# Preparation
sampled_submissions_creators= sampled_submissions.select("subreddit","created_utc","author","title", "score", "num_comments")
sampled_submissions_creators = sampled_submissions_creators.withColumn("score",F.col("score").cast("float"))\
                                         .withColumn("num_comments",F.col("num_comments").cast("float"))\
                                         .withColumn("created_year",F.year(F.to_timestamp(F.col("created_utc").cast('int'))))

#Group submissions data by year and author
sampled_submissions_creators= sampled_submissions_creators.groupBy("created_year","author").agg(
    F.count("*").alias("sub_count"),
    F.sum("score").alias("total_score"),
    F.sum("num_comments").alias("total_num_cmnt"),
    F.round(F.avg("score"),2).alias("avg_score"),
    F.round(F.avg("num_comments"),2).alias("avg_num_cmnt")
).sort('created_year')

# Filter out meaningless data
sampled_submissions_creators= sampled_submissions_creators.filter((sampled_submissions_creators['author']!='[deleted]') &\
                                                                  (sampled_submissions_creators['total_score']!=0) &\
                                                                  (sampled_submissions_creators['total_num_cmnt']!=0))

# Designed for the sample dataset: More straightforward to show the quality of creators' contents for the sample
sampled_submissions_creators= sampled_submissions_creators.withColumn("creator_score",F.col("sub_count")*(F.col("total_score")+F.col("total_num_cmnt")))\
                                                          .orderBy("creator_score",ascending= False)

# Designed for the full dataset: Considering the quantity of comments and scores varies, it's more reasonable to use ranking than simplly adding up the quantity.
# sampled_submissions_creators= sampled_submissions_creators.withColumn("sub_count_ranking",F.dense_rank().over(Window.orderBy("sub_count")))\
#                                                           .withColumn("avg_score_ranking",F.dense_rank().over(Window.orderBy("avg_score")))\
#                                                           .withColumn("avg_num_cmnt_ranking",F.dense_rank().over(Window.orderBy("avg_num_cmnt")))\
#                                                           .withColumn("creator_score",F.col("sub_count_ranking")+F.col("avg_score_ranking")+F.col("avg_num_cmnt_ranking"))\
#                                                           .withColumn('posting_karma',udf_karma(F.col("author")))\
#                                                           .orderBy("creator_score",ascending= False)\

sampled_submissions_creators.cache()

DataFrame[created_year: int, author: string, sub_count: bigint, total_score: double, total_num_cmnt: double, avg_score: double, avg_num_cmnt: double, creator_score: double]

In [12]:
#Generate top10 creators of the year
sampled_creators_top10_2021= sampled_submissions_creators.filter(sampled_submissions_creators['created_year']==2021)\
                                                         .select('created_year',"author","sub_count","avg_score","avg_num_cmnt","creator_score")\
                                                         .limit(10)\
                                                         .withColumn('posting_karma',udf_karma(F.col("author")))\
                                                         .toPandas()
sampled_creators_top10_2022= sampled_submissions_creators.filter(sampled_submissions_creators['created_year']==2022)\
                                                         .select('created_year',"author","sub_count","avg_score","avg_num_cmnt","creator_score")\
                                                         .limit(10)\
                                                         .withColumn('posting_karma',udf_karma(F.col("author")))\
                                                         .toPandas()
sampled_creators_top10_2023= sampled_submissions_creators.filter(sampled_submissions_creators['created_year']==2023)\
                                                         .select('created_year',"author","sub_count","avg_score","avg_num_cmnt","creator_score")\
                                                         .limit(10)\
                                                         .withColumn('posting_karma',udf_karma(F.col("author")))\
                                                         .toPandas()
#Generate top10 creators of the year from 2021 to 2023
sampled_creators_top10_total= pd.concat([sampled_creators_top10_2021,sampled_creators_top10_2022,sampled_creators_top10_2023], axis=0, ignore_index=True)


In [13]:
#Generate top100 creators of the year, proportional to the duration
sampled_creators_top46_2021= sampled_submissions_creators.filter(sampled_submissions_creators['created_year']==2021)\
                                                         .select('created_year',"author","sub_count","avg_score","avg_num_cmnt","creator_score")\
                                                         .limit(46)\
                                                         .withColumn('posting_karma',udf_karma(F.col("author")))\
                                                         .toPandas()
sampled_creators_top46_2022= sampled_submissions_creators.filter(sampled_submissions_creators['created_year']==2022)\
                                                         .select('created_year',"author","sub_count","avg_score","avg_num_cmnt","creator_score")\
                                                         .limit(46)\
                                                         .withColumn('posting_karma',udf_karma(F.col("author")))\
                                                         .toPandas()
sampled_creators_top8_2023= sampled_creators_top10_2023[0:8]
#Generate top100 creators of the year from 2021 to 2023
sampled_creators_top100_total= pd.concat([sampled_creators_top46_2021,sampled_creators_top46_2022,sampled_creators_top8_2023], axis=0, ignore_index=True)


# List of top100 creators from 2021 to 2023, remove duplicate name
sampled_topcreators= sampled_creators_top100_total['author'].to_list()
sampled_topcreators = list(set(sampled_topcreators))

In [14]:
#Generate top100 creators of the year from 2021 to 2023, proportional to the duration
sampled_creators_top46_2021= sampled_submissions_creators.filter(sampled_submissions_creators['created_year']==2021)\
                                                         .select('created_year',"author","sub_count","avg_score","avg_num_cmnt","creator_score")\
                                                         .limit(46)\
                                                         .withColumn('posting_karma',udf_karma(F.col("author")))\
                                                         .toPandas()
sampled_creators_top46_2022= sampled_submissions_creators.filter(sampled_submissions_creators['created_year']==2022)\
                                                         .select('created_year',"author","sub_count","avg_score","avg_num_cmnt","creator_score")\
                                                         .limit(46)\
                                                         .withColumn('posting_karma',udf_karma(F.col("author")))\
                                                         .toPandas()
sampled_creators_top8_2023= sampled_creators_top10_2023[0:8]
sampled_creators_top100_total= pd.concat([sampled_creators_top46_2021,sampled_creators_top46_2022,sampled_creators_top8_2023], axis=0, ignore_index=True)

# List of top100 creators from 2021 to 2023, remove duplicate name
sampled_topcreators= sampled_creators_top100_total['author'].to_list()
sampled_topcreators = list(set(sampled_topcreators))

## New varaibles


In [15]:
# Data preparation
sampled_submissions_var= sampled_submissions.select("author","selftext","created_utc")\
                                        .withColumn("created_hour",F.hour(F.to_timestamp(F.col("created_utc").cast('int'))))\
                                        .withColumn('len',F.length(F.col('selftext')))

# Dummy submissions varaibles
sampled_submissions_var= sampled_submissions_var.withColumn("skincare",F.col("selftext").rlike("""(?i)body|(?i)hair|(?i)facial|(?i)nails|(?i)lip|(?i)sunscreen|(?i)SPF|(?i)acne|(?i)pimples|(?i)scar|(?i)aging"""))\
                                  .withColumn("skincare_product",F.col("selftext").rlike("""(?i)moisturizer|(?i)cleanser|(?i)serum|(?i)toner|(?i)lotion"""))\
                                  .withColumn("skincare_product_brand",F.col("selftext").rlike("""(?i)Clinique|(?i)Neutrogena|(?i)Cetaphil|(?i)Kiehl's|(?i)Olay"""))\
                                  .withColumn("makeup",F.col("selftext").rlike("""(?i)beauty|(?i)bodypaint|(?i)cosmetics|(?i)style|(?i)artist|(?i)cosplay|(?i)fashion|(?i)celebrity|(?i)party|(?i)wedding|(?i)palette"""))\
                                  .withColumn("makeup_product",F.col("selftext").rlike("""(?i)eyeliner|(?i)contour|(?i)foundation|(?i)blush|(?i)lipstick|(?i)concealer"""))\
                                  .withColumn("makeup_product_brand",F.col("selftext").rlike("""(?i)MAC|(?i)NARS|(?i)Sephora|(?i)Fenty|(?i)Revlon|(?i)NYX|(?i)L'Oreal|(?i)Maybelline"""))\
# New submissions varaible
sampled_submissions_var= sampled_submissions_var.withColumn("is_top_100_creator",F.col("author").isin(sampled_topcreators))\
                                        .withColumn("is_peak_hours",F.col("created_hour").isin([16, 17, 18, 19, 20, 21, 22, 23, 0]))\
                                        .withColumn("is_long_text",(F.col('len') > 200))
sampled_submissions_var.cache()

DataFrame[author: string, selftext: string, created_utc: timestamp, created_hour: int, len: int, skincare: boolean, skincare_product: boolean, skincare_product_brand: boolean, makeup: boolean, makeup_product: boolean, makeup_product_brand: boolean, is_top_100_creator: boolean, is_peak_hours: boolean, is_long_text: boolean]

In [16]:
# Data preparation
sampled_comments_var= sampled_comments.select("author","body","created_utc")\
                                  .withColumn("created_hour",F.hour(F.to_timestamp(F.col("created_utc").cast('int'))))\
                                  .withColumn('len',F.length(F.col('body')))
# Dummy comments varaibles
sampled_comments_var= sampled_comments_var.withColumn("skincare",F.col("body").rlike("""(?i)body|(?i)hair|(?i)facial|(?i)nails|(?i)lip|(?i)sunscreen|(?i)SPF|(?i)acne|(?i)pimples|(?i)scar|(?i)aging"""))\
                                  .withColumn("skincare_product",F.col("body").rlike("""(?i)moisturizer|(?i)cleanser|(?i)serum|(?i)toner|(?i)lotion"""))\
                                  .withColumn("skincare_product_brand",F.col("body").rlike("""(?i)Clinique|(?i)Neutrogena|(?i)Cetaphil|(?i)Kiehl's|(?i)Olay"""))\
                                  .withColumn("makeup",F.col("body").rlike("""(?i)beauty|(?i)bodypaint|(?i)cosmetics|(?i)style|(?i)artist|(?i)cosplay|(?i)fashion|(?i)celebrity|(?i)party|(?i)wedding|(?i)palette"""))\
                                  .withColumn("makeup_product",F.col("body").rlike("""(?i)eyeliner|(?i)contour|(?i)foundation|(?i)blush|(?i)lipstick|(?i)concealer"""))\
                                  .withColumn("makeup_product_brand",F.col("body").rlike("""(?i)MAC|(?i)NARS|(?i)Sephora|(?i)Fenty|(?i)Revlon|(?i)NYX|(?i)L'Oreal|(?i)Maybelline"""))\
# New comments varaibles
sampled_comments_var= sampled_comments_var.withColumn("is_top_100_creator",F.col("author").isin(sampled_topcreators))\
                                  .withColumn("is_long_text",(F.col('len') > 200))

sampled_comments_var.cache()

DataFrame[author: string, body: string, created_utc: timestamp, created_hour: int, len: int, skincare: boolean, skincare_product: boolean, skincare_product_brand: boolean, makeup: boolean, makeup_product: boolean, makeup_product_brand: boolean, is_top_100_creator: boolean, is_long_text: boolean]

In [17]:
# Group by and count variables in submissions
sampled_submissions_peak_count= sampled_submissions_var.groupby('is_peak_hours').count().withColumn('is_peak_hours',F.col('is_peak_hours').cast('string')).toPandas()
sampled_submissions_top_count= sampled_submissions_var.groupby('is_top_100_creator').count().withColumn('is_top_100_creator',F.col('is_top_100_creator').cast('string')).toPandas()
sampled_submissions_long_count= sampled_submissions_var.groupby('is_long_text').count().withColumn('is_long_text',F.col('is_long_text').cast('string')).toPandas()
sampled_submissions_skincare_count= sampled_submissions_var.groupby('skincare').count().withColumn('skincare',F.col('skincare').cast('string')).toPandas()
sampled_submissions_makeup_count= sampled_submissions_var.groupby('makeup').count().withColumn('makeup',F.col('makeup').cast('string')).toPandas()
sampled_submissions_var_count= pd.concat([sampled_submissions_peak_count,sampled_submissions_top_count,sampled_submissions_top_count,sampled_submissions_skincare_count,sampled_submissions_makeup_count], axis=1)

# Group by and count variables in comments
sampled_comments_top_count= sampled_comments_var.groupby('is_top_100_creator').count().withColumn('is_top_100_creator',F.col('is_top_100_creator').cast('string')).toPandas()
sampled_comments_long_count= sampled_comments_var.groupby('is_long_text').count().withColumn('is_long_text',F.col('is_long_text').cast('string')).toPandas()
sampled_comments_skincare_count= sampled_comments_var.groupby('skincare').count().withColumn('skincare',F.col('skincare').cast('string')).toPandas()
sampled_comments_makeup_count= sampled_comments_var.groupby('makeup').count().withColumn('makeup',F.col('makeup').cast('string')).toPandas()
sampled_comments_var_count= pd.concat([sampled_comments_top_count,sampled_comments_long_count,sampled_comments_skincare_count,sampled_comments_makeup_count], axis=1)

In [18]:
sampled_submissions_var_count

,is_peak_hours,count,is_top_100_creator,count,is_top_100_creator,count,skincare,count,makeup,count
0,true,4814,true,336,true,336,true,662,true,414
1,false,4920,false,9398,false,9398,false,9072,false,9320


In [19]:
sampled_comments_var_count

,is_top_100_creator,count,is_long_text,count,skincare,count,makeup,count
0,true,2097,true,28912,true,17213,true,15394
1,false,103619,false,76804,false,88503,false,90322
